In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%pip install -q decorator==5.0.9
!pip install -q ipywidgets

import csv
import math
import re
import imp
import json
import numpy as np
import pandas as pd
import difflib as dl
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
from collections import Counter

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q cltk==1.0.22
%pip install -q strsim
%pip install -q leven

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
# importing local modules

import sys
sys.path.append('../src/')

import text_cleanup.text_cleanup as thesisCleanUp
import preprocessing.text_preprocessing as thesisTextPreprocessing
import data.reader as thesisDataReader
import utils.utils as thesisUtils
import features.tf_idf.n_gram as thesisTfIdfNgramFeatures
import features.count_vectorizer.n_gram as thesisCountVectorizerNgramFeatures
import similarities.cosine as thesisCosineSimilarities
import similarities.levenshtein as thesisLevenshteinSimilarities
import vocabulary.vocabulary as thesisVocabulary
import features.lexical as thesisLexicalFeatures
import similarities.cosine as thesisCosineSimilarity
import text_cleanup.text_cleanup as thesisTextCleanUp
import p_aligment.p_aligment as thesisPAligment
import features.model_features as thesisModelFeatures

imp.reload(thesisTfIdfNgramFeatures)
imp.reload(thesisLexicalFeatures)
imp.reload(thesisCosineSimilarity)
imp.reload(thesisCleanUp)
imp.reload(thesisTextPreprocessing)
imp.reload(thesisDataReader)
imp.reload(thesisUtils)

imp.reload(thesisVocabulary)
imp.reload(thesisCosineSimilarities)
imp.reload(thesisTextCleanUp)
imp.reload(thesisCountVectorizerNgramFeatures)
imp.reload(thesisPAligment)
imp.reload(thesisLevenshteinSimilarities)
imp.reload(thesisModelFeatures)

<module 'features.model_features' from '../src/features/model_features.py'>

# Create burchard version in original london format

1. create london/zwickau versions without words/letters replacement - replace only numbers, punctuation, etc...
2. create word mapping of a new created versions in 1 to regular processed versions
3. change words in burchard candidate according to mapping in 2 - it should return us burchard in original format

In [5]:
london_corpus_by_new_line_without_words_processing = thesisDataReader.get_london_by_new_line_without_words_processing()
zwickau_corpus_by_new_line_without_words_processing = thesisDataReader.get_zwickau_by_new_line_without_words_processing()

In [6]:
london_original_to_processed_mapping = thesisVocabulary.create_pre_post_processing_map(
    ' \n'.join(london_corpus_by_new_line_without_words_processing)
)

In [7]:
burchard_version_with_original_london_text = []

for p in thesisDataReader.get_burchard_candidate_version_based_on_strongly_similar_london_base():
    new_p = ''
    
    for word in p.split():
        original_word = london_original_to_processed_mapping[london_original_to_processed_mapping['after'] == word]['before'].values[0]
        new_p = f'{new_p} {original_word}' if len(new_p) > 0 else original_word
    
    burchard_version_with_original_london_text.append(new_p)

# td_idf_3_gram + cosine_similarity

In [8]:
features_tfidf_3_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 3,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

n_gram_feature_name: 3_gram
n_gram_feature_name: 3_gram


In [9]:
features_tfidf_3_gram_cosine_similarity_df

,index,corpus_version_label,a,aa,ab,ac,ad,ae,af,ag,...,zoc,zoe,zor,zot,zra,zuc,zur,zyd,zyp,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.065368
1,1.0,0.0,0.0,0.05966,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.268126
2,2.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.209745
3,3.0,0.0,0.0,0.00000,0.000000,0.000000,0.028476,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.243578
4,4.0,0.0,0.0,0.00000,0.021612,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.230167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.00000,0.000000,0.032224,0.033061,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.198400
634,317.0,1.0,0.0,0.00000,0.013060,0.017769,0.036460,0.033796,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.311705
635,318.0,1.0,0.0,0.00000,0.000000,0.000000,0.023401,0.000000,0.0,0.021638,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.241369
636,319.0,1.0,0.0,0.00000,0.027644,0.000000,0.057881,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.210448


In [10]:
tf_idf_3_cosine_results = thesisModelFeatures.run_models(features_tfidf_3_gram_cosine_similarity_df)

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

running: SVM_RBF
running: DecisionTreeClassifier
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

running: RandomForestClassifier
running: MLPClassifier
running: GaussianNB
running: KNeighborsClassifier
running: AdaBoostClassifier
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

In [12]:
tf_idf_3_cosine_results[0].sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.615261,0.611981,0.609936,0.612698,0.610244,0.612698
MLPClassifier,0.592646,0.594674,0.587741,0.595263,0.588115,0.595263
GaussianNB,0.566984,0.560568,0.551019,0.559549,0.550568,0.559549
DecisionTreeClassifier,0.552385,0.551523,0.549492,0.551711,0.549616,0.551711
RandomForestClassifier,0.547681,0.543316,0.537544,0.543750,0.537829,0.543750
QuadraticDiscriminantAnalysis,0.507707,0.506960,0.499495,0.507614,0.499896,0.507614
SVM_linear,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianProcessClassifier,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
SVM_RBF,0.485271,0.485456,0.475259,0.485838,0.475505,0.485838
KNeighborsClassifier,0.469402,0.469911,0.464243,0.469916,0.464283,0.469916


# td_idf_4_gram + cosine_similarity


In [13]:
features_tfidf_4_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 4,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

n_gram_feature_name: 4_gram
n_gram_feature_name: 4_gram


In [14]:
features_tfidf_4_gram_cosine_similarity_df

,index,corpus_version_label,a a,a b,a c,a d,a f,a g,a i,a l,...,zra,zrae,zrah,zuca,zucc,zucu,zuri,zyda,zyph,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021962
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.086926
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.066421
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083912
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.072329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062225
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118377
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.087864
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062714


In [15]:
tf_idf_4_cosine_results = thesisModelFeatures.run_models(features_tfidf_4_gram_cosine_similarity_df)

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

running: SVM_RBF
running: DecisionTreeClassifier
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

running: RandomForestClassifier
running: MLPClassifier
running: GaussianNB
running: KNeighborsClassifier
running: AdaBoostClassifier
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

In [16]:
tf_idf_4_cosine_results[0].sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.605299,0.601037,0.597845,0.601711,0.598154,0.601711
DecisionTreeClassifier,0.601301,0.599655,0.594942,0.600025,0.595203,0.600025
MLPClassifier,0.573728,0.573250,0.567882,0.573487,0.568071,0.573487
RandomForestClassifier,0.566967,0.565585,0.563515,0.565625,0.563583,0.565625
GaussianNB,0.570435,0.557241,0.545683,0.556275,0.545273,0.556275
SVM_linear,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianProcessClassifier,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
SVM_RBF,0.493824,0.492317,0.468826,0.493601,0.469575,0.493601
QuadraticDiscriminantAnalysis,0.468532,0.469249,0.465265,0.468775,0.465059,0.468775
KNeighborsClassifier,0.465489,0.468298,0.460295,0.468626,0.460511,0.468626


# td_idf_5_gram + cosine_similarity

In [17]:
features_tfidf_5_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 5,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

n_gram_feature_name: 5_gram
n_gram_feature_name: 5_gram


In [18]:
features_tfidf_5_gram_cosine_similarity_df

,index,corpus_version_label,a aq,a ba,a be,a ca,a ce,a ch,a ci,a cr,...,zra d,zrael,zrahe,zucar,zucca,zucur,zurio,zyda,zyph,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009855
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032757
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025986
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033445
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024990
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050199
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034937
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023450


In [19]:
tf_idf_5_cosine_results = thesisModelFeatures.run_models(features_tfidf_5_gram_cosine_similarity_df)

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

running: SVM_RBF
running: DecisionTreeClassifier
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

running: RandomForestClassifier
running: MLPClassifier
running: GaussianNB
running: KNeighborsClassifier
running: AdaBoostClassifier
running: QuadraticDiscriminantAnalysis


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analy

In [20]:
tf_idf_5_cosine_results[0].sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.597362,0.596731,0.595694,0.596875,0.595811,0.596875
DecisionTreeClassifier,0.596147,0.595109,0.592871,0.595412,0.593054,0.595412
MLPClassifier,0.579352,0.580366,0.574947,0.581200,0.575406,0.581200
RandomForestClassifier,0.560599,0.556682,0.550078,0.557738,0.550626,0.557738
GaussianNB,0.534173,0.512181,0.491763,0.510913,0.491149,0.510913
SVM_linear,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianProcessClassifier,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
KNeighborsClassifier,0.495764,0.498190,0.491562,0.498264,0.491655,0.498264
QuadraticDiscriminantAnalysis,0.489654,0.489716,0.482050,0.489087,0.481766,0.489087
SVM_RBF,0.422899,0.456739,0.354880,0.459152,0.356477,0.459152


# td_idf_6_gram + cosine_similarity

In [21]:
features_tfidf_6_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 6,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

n_gram_feature_name: 6_gram
n_gram_feature_name: 6_gram


In [22]:
features_tfidf_6_gram_cosine_similarity_df

,index,corpus_version_label,a aqu,a bab,a bel,a ber,a bet,a car,a cas,a ced,...,zra di,zrael,zrahel,zucara,zuccar,zucura,zurio,zyda o,zyph d,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005269
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014145
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011290
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015839
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011393
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023521
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015032
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010123


In [27]:
tf_idf_6_cosine_results = thesisModelFeatures.run_models(features_tfidf_6_gram_cosine_similarity_df)

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

running: SVM_RBF


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


running: DecisionTreeClassifier
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


running: RandomForestClassifier
running: MLPClassifier
running: GaussianNB
running: KNeighborsClassifier
running: AdaBoostClassifier


In [28]:
tf_idf_6_cosine_results[0].sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.597744,0.597998,0.594211,0.598413,0.594495,0.598413
MLPClassifier,0.571864,0.573048,0.569497,0.573462,0.569772,0.573462
RandomForestClassifier,0.564561,0.567317,0.557299,0.568601,0.558005,0.568601
DecisionTreeClassifier,0.558155,0.559068,0.553301,0.559276,0.553497,0.559276
KNeighborsClassifier,0.521402,0.522755,0.516872,0.523264,0.517151,0.523264
SVM_linear,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianProcessClassifier,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianNB,0.511939,0.502560,0.482621,0.501463,0.482112,0.501463
SVM_RBF,0.247379,0.439299,0.308345,0.441915,0.310224,0.441915


# td_idf_7_gram + cosine_similarity

In [29]:
features_tfidf_7_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 7,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

n_gram_feature_name: 7_gram
n_gram_feature_name: 7_gram


In [30]:
features_tfidf_7_gram_cosine_similarity_df

,index,corpus_version_label,a aque,a baby,a belu,a bers,a beth,a carm,a cast,a casu,...,zrahel,zraheli,zucara,zuccara,zucura,zurio e,zurio i,zyda op,zyph di,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002966
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006549
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005263
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008438
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005794
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.011862
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006710
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004590


In [31]:
tf_idf_7_cosine_results = thesisModelFeatures.run_models(features_tfidf_7_gram_cosine_similarity_df)

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

running: SVM_RBF


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


running: DecisionTreeClassifier
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings

running: RandomForestClassifier
running: MLPClassifier
running: GaussianNB
running: KNeighborsClassifier
running: AdaBoostClassifier


In [32]:
tf_idf_7_cosine_results[0].sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
AdaBoostClassifier,0.614764,0.613421,0.608981,0.614038,0.609330,0.614038
DecisionTreeClassifier,0.578233,0.577619,0.568320,0.578150,0.568662,0.578150
RandomForestClassifier,0.572167,0.565408,0.542596,0.567138,0.543515,0.567138
MLPClassifier,0.551585,0.552629,0.548445,0.553075,0.548739,0.553075
KNeighborsClassifier,0.532461,0.532171,0.530163,0.532738,0.530482,0.532738
SVM_linear,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianProcessClassifier,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianNB,0.504462,0.497724,0.482623,0.496900,0.482282,0.496900
SVM_RBF,0.236350,0.443987,0.308346,0.446602,0.310224,0.446602


# td_idf_8_gram + cosine_similarity

In [33]:
features_tfidf_8_gram_cosine_similarity_df = thesisModelFeatures.create_features_df(
    london_corpus_by_new_line_without_words_processing,
    zwickau_corpus_by_new_line_without_words_processing,
    None,
    n_gram = 8,
    features = {'tfidf', 'inner_mean_cosine_similarity_score'}
#     burchard_version_with_original_london_text
)

n_gram_feature_name: 8_gram
n_gram_feature_name: 8_gram


In [34]:
features_tfidf_8_gram_cosine_similarity_df

,index,corpus_version_label,a aque,a babyl,a belua,a bersa,a betha,a bethe,a beths,a bethu,...,zraheli,zrahelit,zucara m,zuccara,zucura e,zurio et,zurio it,zyda opi,zyph dic,inner_mean_cosine_similarity_score
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001788
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003370
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002508
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004578
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,316.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003071
634,317.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006157
635,318.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003205
636,319.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002167


In [35]:
tf_idf_8_cosine_results = thesisModelFeatures.run_models(features_tfidf_8_gram_cosine_similarity_df)

running: SVM_linear


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaco

running: SVM_RBF


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


running: DecisionTreeClassifier
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings

running: RandomForestClassifier
running: MLPClassifier
running: GaussianNB
running: KNeighborsClassifier
running: AdaBoostClassifier


In [36]:
tf_idf_8_cosine_results[0].sort_values(by=['accuracy'], ascending=False)

,precision_macro,recall_macro,f1_macro,f1_micro,f1_weighted,accuracy
DecisionTreeClassifier,0.611469,0.610050,0.604008,0.610913,0.604472,0.610913
AdaBoostClassifier,0.578964,0.574062,0.564160,0.574975,0.564666,0.574975
RandomForestClassifier,0.543276,0.547666,0.509748,0.549826,0.511053,0.549826
MLPClassifier,0.538136,0.539987,0.535850,0.540451,0.536153,0.540451
KNeighborsClassifier,0.521140,0.521183,0.518384,0.521850,0.518767,0.521850
SVM_linear,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianProcessClassifier,0.251575,0.500000,0.334722,0.503150,0.336855,0.503150
GaussianNB,0.504350,0.503820,0.485701,0.502927,0.485255,0.502927
SVM_RBF,0.237257,0.447112,0.309877,0.449727,0.311755,0.449727


# tf_idf_5_gram + cosine similarity GridSearchCV

In [39]:
resp = []
for cls in ['SVC', 'DecisionTreeClassifier', 'GaussianProcessClassifier', 'RandomForestClassifier', 'GaussianNB', 'KNeighborsClassifier', 'AdaBoostClassifier']:
    grid_search_cv_result = thesisModelFeatures.run_grid_search_cv(features_tfidf_5_gram_cosine_similarity_df, [cls])
    resp.append([cls, grid_search_cv_result[1][0].best_score_])
    print(grid_search_cv_result[1][0].best_score_)

testing classifiers: ['SVC']
running: SVC
0.6235863095238094
testing classifiers: ['DecisionTreeClassifier']
running: DecisionTreeClassifier
0.640798611111111
testing classifiers: ['GaussianProcessClassifier']
running: GaussianProcessClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


0.5031498015873016
testing classifiers: ['RandomForestClassifier']
running: RandomForestClassifier


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.5812748  0.6030506  0.6031498  0.6094246  0.6140873  0.6250744
 0.6078125  0.6110119  0.5812748  0.6030506  0.6031498  0.6094246
 0.6140873  0.6250744  0.6078125  0.6110119  0.51386409 0.52333829
 0.54047619 0.53578869 0.55458829 0.5452877  0.5452877  0.5390873
 0.57641369 0.6030506  0.59987599 0.61875    0.6125496  0.6156746
 0.6109375  0.6109623  0.57641369 0.6030506  0.59987599 0.61875
 0.6125496  0.6156746  0.6109375  0.6109623  0.54052579 0.55143849
 0.55146329 0.5453621  0.55625    0.5593998  0.5484623  0.5687996
 0.5999752  0.6109127  0.6077629  0.6031002  0.6046875  0.61884921
 0.61416171 0.6203621  0.5999752  0.6109127  0.6077629  0.6031002
 0.6046875  0.61884921 0.61416171 0.6203621  0.51855159 0.52936508
 0.53412698 0.5562748  0.5546131  0.54990079 0.5484127  0.5562004
 0.5702629  0.6015129  0.5985119  0.604761

0.6422867063492064
testing classifiers: ['GaussianNB']
running: GaussianNB


/Users/sasha.kruglyak/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


0.5374751984126984
testing classifiers: ['KNeighborsClassifier']
running: KNeighborsClassifier
0.5482390873015872
testing classifiers: ['AdaBoostClassifier']
running: AdaBoostClassifier
0.6642609126984127


In [40]:
resp

[['SVC', 0.6235863095238094],
 ['DecisionTreeClassifier', 0.640798611111111],
 ['GaussianProcessClassifier', 0.5031498015873016],
 ['RandomForestClassifier', 0.6422867063492064],
 ['GaussianNB', 0.5374751984126984],
 ['KNeighborsClassifier', 0.5482390873015872],
 ['AdaBoostClassifier', 0.6642609126984127]]